In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import shap
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# -----------------------------
# Load trained model
# -----------------------------
model = tf.keras.models.load_model(
    "/kaggle/input/mymodel/inceptionv3_model1.keras"
)

# -----------------------------
# Configuration
# -----------------------------
IMG_SIZE = (224, 224)
test_folder = "/kaggle/input/testdata/stomachTest"

# -----------------------------
# Load sample test images
# -----------------------------
images = []
labels = []
class_names = sorted(os.listdir(test_folder))

for class_name in class_names:
    class_path = os.path.join(test_folder, class_name)
    for img_name in os.listdir(class_path)[:2]:  # 2 images per class
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path).convert("RGB")
        img = img.resize(IMG_SIZE)
        img_array = np.array(img) / 255.0
        images.append(img_array)
        labels.append(class_name)

images = np.array(images)

print("Images shape:", images.shape)

# -----------------------------
# Prediction function (VERY IMPORTANT)
# -----------------------------
def predict_fn(x):
    return model.predict(x)

# -----------------------------
# SHAP masker & explainer (STABLE)
# -----------------------------
masker = shap.maskers.Image("blur(64,64)", images[0].shape)

explainer = shap.Explainer(
    predict_fn,
    masker,
    output_names=class_names
)

# -----------------------------
# Compute SHAP values
# -----------------------------
shap_values = explainer(images, max_evals=100)

# -----------------------------
# Plot explanations
# -----------------------------
shap.image_plot(shap_values, images)
